In [28]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.constants import M_sun, G
import astropy.units as u
import tqdm
from matplotlib import colors 
from scipy.integrate import simps
from scipy.interpolate import interp1d

sys.path.append('../my_funcs/')
from pipeline_main import pipeline, calc_ang
from pipeline_1D import *


# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 16}
mpl.rc('font', **font)

In [29]:
datai = pipeline(snap = 446, run = 'sink225core03', sink_id=225)
data = pipeline(snap = 524, run = 'sink225core03', sink_id=225)
data.recalc_L()

Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4903/4903 [00:06<00:00, 800.23it/s]


Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4903/4903 [00:05<00:00, 834.74it/s]


Converged mean angular momentum vector after 0 iteration(s)


In [30]:
data.to_1D(plot = False, r_in= 5, r_out = 100, Nr = 100)

Looping through patches to extract densities and heights
Fitting surface density and scaleheight in each radial bin


100%|██████████| 99/99 [00:03<00:00, 26.78it/s] 


In [31]:
data.get_1D_param(Ω = True)

Extracting azimuthal, angular velocities and adiabatic index data from patches into 1D


100%|██████████| 1803/1803 [00:04<00:00, 435.53it/s]

Σ = g/cm2
φ = rad
H = au
vφ = cm/s
Ω = 1/s
cs = cm/s
Q = dimensionless
B = Gauss
T = K


# $$L_{tot} = 2\pi\int\Sigma v_\phi r^2 dr$$ 2 orders of magnitude higher
# $$L_{tot} = \sum_i r_i\times m_i v_i$$

In [32]:
H = data.H_1D[:,0]
r = data.r_1D * data.sn.scaling.l
Σ = data.Σ_1D[:,0]
v_φ = data.vφ_1D[:,0]
v_kep = data.kepVφ_1D

In [33]:
simps(2 * np.pi * Σ * v_φ * r**2, r) 

2.1102584358666373e+51

In [34]:
simps(2 * np.pi * Σ * v_kep * r**2, r) 

2.5266995507714567e+51

In [35]:
simps(2 * np.pi * Σ * v_φ * r**2, r) * data.cgs_AM / (simps(2 * np.pi * Σ * v_kep * r**2, r) * data.cgs_AM)

0.8351837618454989

In [36]:
H_func = interp1d(r, H / data.au_length, fill_value='extrapolate')

In [37]:
for p in data.sorted_patches1D:
    if p.id == 1853:break

In [38]:
L_new = np.zeros(3)
for p in data.sorted_patches1D:
        nbors = [data.sn.patchid[i] for i in p.nbor_ids if i in data.sn.patchid]
        children = [ n for n in nbors if n.level == p.level + 1]
        leafs = [n for n in children if ((n.position - p.position)**2).sum() < ((p.size)**2).sum()/12]   
        if len(leafs) == 8: continue

        to_extract = (p.cyl_R > 3 / data.au_length) & (p.cyl_R < 100 / data.au_length) & (3 *H_func(p.cyl_R) > abs(p.cyl_z)) #### CHANGE 3 AND 100!!!!############

        if np.sum(to_extract) == 0: continue
        for lp in leafs: 
            leaf_extent = np.vstack((lp.position - 0.5 * lp.size, lp.position + 0.5 * lp.size)).T
            covered_bool = ~np.all((p.xyz > leaf_extent[:, 0, None, None, None]) & (p.xyz < leaf_extent[:, 1, None, None, None]), axis=0)
            to_extract *= covered_bool 

        L_new += np.cross(p.rel_xyz, (p.vel_xyz * p.m) , axisa=0, axisb=0, axisc=0)[:,to_extract].sum(axis = 1)
        
        
total_L = np.linalg.norm(L_new)


In [39]:
L_new /total_L, data.L

/tmp/ipykernel_3521271/3116311928.py:1: RuntimeWarning: invalid value encountered in true_divide
  L_new /total_L, data.L


(array([nan, nan, nan]), array([-0.26901008,  0.64655394,  0.71386384]))

In [40]:
calc_ang(L_new /total_L, data.L)

/tmp/ipykernel_3521271/3338556159.py:1: RuntimeWarning: invalid value encountered in true_divide
  calc_ang(L_new /total_L, data.L)


nan

In [41]:
data.total_L * data.cgs_AM, total_L * data.cgs_AM

(1.8204138608098663e+52, 0.0)

In [42]:
L_new, data.L

(array([0., 0., 0.]), array([-0.26901008,  0.64655394,  0.71386384]))

# Now doing it to see the time evolution of $L/L_{kep}$

In [48]:
snapshots = np.arange(datai.sn.iout, data.sn.iout)
dict_to_save = {key: np.zeros_like(snapshots, dtype=float) for key in ['disk_size', 'L_tot', 'L_kep', 'time']}

In [52]:
dict_to_save = {key: np.zeros_like(snapshots) for key in ['disk_size', 'L_tot', 'L_kep', 'time']}
for snaps_j in tqdm.tqdm(range(len(snapshots))):

    data_loop = pipeline(snap = datai.sn.iout + snaps_j , run = 'sink225core03', sink_id = 225, loading_bar = False, verbose = 0)
    data_loop.recalc_L(verbose = 0)
    data_loop.calc_disksize(verbose=0, plot = False)

    dict_to_save['disk_size'][snaps_j] = np.copy(data_loop.disk_size)

    data_loop.to_1D(plot = False, r_in= 5, r_out = 100, Nr = 100, verbose = 0)
    data_loop.get_1D_param(Ω = True, verbose = 0, get_units=False)
    H = data_loop.H_1D[:,0]
    r = data_loop.r_1D * data.sn.scaling.l
    Σ = data_loop.Σ_1D[:,0]
    v_φ = data_loop.vφ_1D[:,0]
    v_kep = data_loop.kepVφ_1D

    L_tot = simps(2 * np.pi * Σ * v_φ * r**2, r)
    L_kep = simps(2 * np.pi * Σ * v_kep * r**2, r)  
    dict_to_save['L_tot'][snaps_j] = L_tot * 1e-50
    dict_to_save['L_kep'][snaps_j] = L_kep * 1e-50
    dict_to_save['time'][snaps_j] = data_loop.time - datai.time + 500


  0%|          | 0/78 [00:00<?, ?it/s]

446


  0%|          | 0/78 [00:33<?, ?it/s]


In [47]:
type(L_tot)

numpy.float64

In [51]:
dict_to_save['L_tot'][0] = L_tot

OverflowError: Python int too large to convert to C long

In [ ]:
simps(2 * np.pi * Σ * v_kep * r**2, r) 

-2.5465934746291788e+51

In [ ]:
Σ

array([ 2.10663408e+01,  3.66559877e+03, -2.42247309e+01, -1.97428258e+01,
       -1.45992185e+04,  3.27498070e+01,  9.06915427e+01,  1.97304025e+04,
        3.66182784e+01,  2.95543984e+01,  2.74839793e+01,  3.42396480e+01,
        3.11352634e+01,  8.53324918e+03,  6.36789378e+03,  2.62755435e+01,
        2.10572645e+01,  5.89836952e+03, -3.64444249e+01, -2.37988061e+01,
       -1.85589642e+01, -2.07605084e+01, -1.73364491e+01, -1.93894929e+01,
       -1.67388030e+01, -1.79290156e+01, -1.56222098e+01, -1.63267580e+01,
       -1.63297579e+01, -1.55436569e+01, -1.58933542e+01, -1.50370756e+01,
       -1.66313529e+01, -1.57759052e+01, -1.62182317e+01, -1.58326844e+01,
       -1.85381089e+01, -1.67128115e+01, -1.65550053e+01, -1.74442091e+01,
       -1.54988275e+01, -1.52323056e+01, -1.43777044e+01, -1.36948703e+01,
       -1.33470441e+01, -1.24523459e+01, -1.18800894e+01, -1.14827721e+01,
       -1.09284846e+01, -1.04892765e+01, -1.01364951e+01, -9.90084337e+00,
       -1.00294301e+01, -

In [ ]:
simps(2 * np.pi * Σ * v_φ * r**2, r)

nan

In [ ]:
np.savetxt('data_for_plotting/L_evo.txt', L_evo)

array([500,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
snapshots

array([446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458,
       459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471,
       472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484,
       485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497,
       498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510,
       511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523])